In [86]:
pip install requests beautifulsoup4 markdownify

^C
Fatal Python error: init_import_size: Failed to import the site module
Python runtime state: initialized
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site.py", line 73, in <module>
    import os
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/os.py", line 59, in <module>
    import posixpath as path
  File "<frozen importlib._bootstrap>", line 991, in _find_and_load
  File "<frozen importlib._bootstrap>", line 975, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 671, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 844, in exec_module
  File "<frozen importlib._bootstrap_external>", line 939, in get_code
  File "<frozen importlib._bootstrap_external>", line 1037, in get_data
KeyboardInterrupt
Note: you may need to restart the kernel to use updated packages.


In [101]:
import requests
from bs4 import BeautifulSoup as bs
import markdownify
import time

for page_no in range(5, 148 + 1):
    # time.sleep(2)

    print(page_no, ' ... ', end='')
    html = requests.get(f'https://hyostech.tistory.com/{page_no}')

    if html.status_code == 404:
        print('404')
        continue

    soup = bs(html.text, "html.parser")

    title = soup.select_one(".article-header .title-article")
    print(title.text, ' ... ', end='')

    content = soup.select_one(".article-view .contents_style")
    content_md = markdownify.markdownify(str(content), heading_style="ATX")
    # print(content_md[:10], ' ... ')
    # print(content_md)

    category = soup.select_one(".article-header .category")
    # print(category.text)

    tags = soup.select(".article-tag .box-tag a")
    tags = '\n  '.join(list(map(lambda tag: f'- {tag.text}', tags)))

    date = soup.select_one(".article-header .date")
    year, month, day, hm = date.text.split('.')
    hour, minute = hm.split(':')
    # print(year, month, day, hour, minute)

    md_base_name = f'{year.strip()}-{month.strip()}-{day.strip()}-{hour.strip()}-{minute.strip()}'
    image_dir = os.path.join('assets/images/posts', md_base_name)

    if images:
        os.makedirs(image_dir, exist_ok=True)
    for n, image in enumerate(content.select("img"), start=1):
        src = image['src']
        # print(src)
        image_path = os.path.join(image_dir, f'img-{n}.png')
        open(image_path, 'wb').write(requests.get(src).content)
        content_md = content_md.replace(src, f'/{image_path}')

    metadata = f"""---
title: {title.text}
toc: true
categories:
  - {category.text}
tags:
  {tags}
---

    """
    content_md = metadata + content_md
    open(os.path.join('_posts', f'{md_base_name}.md'), 'w').write(content_md)

    print('OK')

5  ... Windows에 ELK Stack 구축 (셋업)  ... OK
6  ... 데이터 스크래핑 1 (BeautifulSoup)  ... OK
7  ... Windows에 ELK Stack 구축 2 (데이터 수집 및 적재)  ... OK
8  ... Windows에 ELK Stack 구축 3 (시각화)  ... OK
9  ... 404
10  ... 데이터 스크래핑 2 (Selenium)  ... OK
11  ... 데이터 스크래핑 3 (API 추출, without Selenium)  ... OK
12  ... X-Pack Security 적용 (Elastic Stack 8.1 기준)  ... OK
13  ... 데이터 파이프라인 핵심 가이드  ... OK
14  ... Elasticsearch 외부 공개  ... OK
15  ... 기초부터 다지는 ElasticSearch 운영 노하우  ... OK
16  ... Windows에 ELK Stack 구축 4 (형태소 분석, Word Cloud)  ... OK
17  ... 스프링 부트와 AWS로 혼자 구현하는 웹 서비스  ... OK
18  ... Airflow 2.3 설치 (w. WSL2)  ... OK
19  ... Airflow v2.2 이상 schedule 관련 용어  ... OK
20  ... Airflow에서 PyTorch 사용 (KubernetesPodOperator)  ... OK
21  ... PyTorch image에서 apt update 실패 (NO_PUBKEY)  ... OK
22  ... Elasticsearch 성능 최적화  ... OK
23  ... Tetra - Full stack component framework  ... OK
24  ... Docker dangling image (none) 제거  ... OK
25  ... KubernetesPodOperator를 이용해서 GPU 할당 받기  ... OK
26  ... BentoML 설치 및 동작 확인  ... OK
27